<a href="https://colab.research.google.com/github/ammtjm/Stock_Data_Analysis/blob/main/SP500%E5%8F%AF%E8%A6%96%E5%8C%96%E3%83%87%E3%83%BC%E3%82%BF%E3%81%9D%E3%81%AE1pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 必要なライブラリをインストール
!pip install yfinance

import yfinance as yf
import datetime

# 現在の日付を取得
end_date = datetime.datetime.now()

# 15年前の日付を計算
start_date = end_date - datetime.timedelta(days=15*365)

# S&P 500指数のデータを取得
sp500 = yf.download('^GSPC', start=start_date, end=end_date)

# データフレームの最初の5行を表示
print(sp500.head())


[*********************100%%**********************]  1 of 1 completed
                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2008-09-29  1209.069946  1209.069946  1106.420044  1106.420044  1106.420044   
2008-09-30  1113.780029  1168.030029  1113.780029  1166.359985  1166.359985   
2008-10-01  1164.170044  1167.030029  1140.770020  1161.060059  1161.060059   
2008-10-02  1160.640015  1160.640015  1111.430054  1114.280029  1114.280029   
2008-10-03  1115.160034  1153.819946  1098.140015  1099.229980  1099.229980   

                Volume  
Date                    
2008-09-29  7305060000  
2008-09-30  4937680000  
2008-10-01  5782130000  
2008-10-02  6285640000  
2008-10-03  6716120000  


In [ ]:
sp500.to_csv("sp500指数ヒストリカルデータ.csv")

In [ ]:
!pip install Dash

In [ ]:
import dash
from dash import html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import numpy as np
import dash_table

import pandas as pd
import numpy as np
import scipy.stats as stats
from dash import Dash, html, dcc, Output, Input
import plotly.express as px
import plotly.figure_factory as ff
from scipy.stats import shapiro


# ファイルからデータを読み込む
file_path = 'sp500指数ヒストリカルデータ.csv'  # ファイルパスを適切に設定してください
df = pd.read_csv(file_path)
df['Date'] = pd.to_datetime(df['Date'])

# 指標の計算
df['LogReturn'] = np.log(df['Close'] / df['Close'].shift(1))
df['CumLogReturn'] = df['LogReturn'].cumsum()
df['PctReturn'] = df['Close'].pct_change()
df['CumPctReturn'] = (1 + df['PctReturn']).cumprod() - 1
df['SMA_20'] = df['Close'].rolling(window=20).mean()
df['SMA_100'] = df['Close'].rolling(window=100).mean()
df['weekday'] = df['Date'].dt.weekday
df['month'] = df['Date'].dt.month

# Dashアプリケーションの作成
app = dash.Dash(__name__)

# アプリケーションのレイアウト定義
app.layout = html.Div([
    dcc.Tabs(id='tabs', value='tab1', children=[
        dcc.Tab(label='基本統計量', value='tab1'),
        dcc.Tab(label='移動平均と終値のプロット', value='tab2'),
        dcc.Tab(label='終値と取引量のプロット', value='tab3'),
        dcc.Tab(label='リターンの分布', value='tab4'),
        dcc.Tab(label='累積対数リターンのヒストグラム', value='tab5'),
        dcc.Tab(label='リターンの箱ひげ図', value='tab6'),
        dcc.Tab(label='曜日と月別のリターンのヒートマップ', value='tab7'),
        dcc.Tab(label='マーケットオープン時とクローズ時のリターンの比較', value='tab8'),
        dcc.Tab(label='マーケットオープン時とクローズ時のリターンのヒストグラム', value='tab9'),
    ]),
    html.Div(id='tabs-content')
])

# コールバック定義：タブの選択に応じて表示するコンテンツを切り替える
@app.callback(Output('tabs-content', 'children'),
              Input('tabs', 'value'))
def render_content(tab):
    # タブ1の内容
    if tab == 'tab1':
    # 基本統計量の計算
      stats_summary = df.describe().transpose().reset_index()
      stats_summary.columns = ['Feature', 'Count', 'Mean', 'Std', 'Min', '25%', '50%', '75%', 'Max']

    # 欠損率の計算
      missing_rate = df.isnull().mean().reset_index()
      missing_rate.columns = ['Feature', 'Missing Rate']

    # 結果の表示
      return html.Div([
        html.H3('データの基本統計量'),
        dash_table.DataTable(
            data=stats_summary.to_dict('records'),
            columns=[{'name': i, 'id': i} for i in stats_summary.columns]
        ),
        html.H3('データの欠損率'),
        dash_table.DataTable(
            data=missing_rate.to_dict('records'),
            columns=[{'name': i, 'id': i} for i in missing_rate.columns]
        )
    ])

    elif tab == 'tab2':
        return html.Div([
            dcc.Graph(
                figure=go.Figure(
                    data=[
                        go.Scatter(x=df['Date'], y=df['SMA_20'], name='SMA_20'),
                        go.Scatter(x=df['Date'], y=df['SMA_100'], name='SMA_100'),
                        go.Scatter(x=df['Date'], y=df['Close'], name='Close')
                    ],
                    layout=go.Layout(title='移動平均と終値のプロット', xaxis=dict(title='Year'), yaxis=dict(title='Value'))
                )
            )
        ])
    elif tab == 'tab3':
        return html.Div([
            dcc.Graph(
                figure=go.Figure(
                    data=[
                        go.Scatter(x=df['Date'], y=df['Close'], yaxis='y1', name='Close'),
                        go.Bar(x=df['Date'], y=df['Volume'], yaxis='y2', name='Volume', opacity=1)
                    ],
                    layout=go.Layout(
                        title='終値と取引量のプロット',
                        xaxis=dict(title='Year'),
                        yaxis=dict(title='Close'),
                        yaxis2=dict(title='Volume', overlaying='y', side='right')
                    )
                )
            )
            ])
    elif tab == 'tab4':
    # リターンのヒストグラム
      hist_data_return = go.Histogram(
          x=df["PctReturn"],
          opacity=0.75,
          name='ただのリターン',
          histnorm='probability density',
          nbinsx=100
      )

      # 対数リターンのヒストグラム
      hist_data_log_return = go.Histogram(
          x=df["LogReturn"],
          opacity=0.75,
          name='対数変換したリターン',
          histnorm='probability density',
          nbinsx=100
      )

      # 正規分布の理論値
      x = np.linspace(min(df["LogReturn"].min(), df["PctReturn"].min()), max(df["LogReturn"].max(), df["PctReturn"].max()), 1000)
      y = norm.pdf(x, 0, 1)

      normal_dist = go.Scatter(
          x=x,
          y=y,
          mode='lines',
          name='正規分布の理論値'
      )

      # レイアウトの設定
      layout = go.Layout(
          title='リターンと対数リターンのヒストグラムと正規分布の理論値',
          xaxis=dict(title='Value'),
          yaxis=dict(title='Probability Density')
      )

      return html.Div([
          dcc.Graph(
              figure=go.Figure(
                  data=[hist_data_return, hist_data_log_return, normal_dist],
                  layout=layout
              )
        )
    ])


    elif tab == 'tab5':
        return html.Div([
            dcc.Graph(
                figure=px.histogram(df, x="CumLogReturn", nbins=100, title='累積対数リターンのヒストグラム')
            )
        ])
    elif tab == 'tab6':
        return html.Div([
            dcc.Graph(
                figure=px.box(df, x="LogReturn", title='リターンの箱ひげ図')
            )
        ])
    elif tab == 'tab7':
        df_pivot = df.pivot_table(index='weekday', columns='month', values='LogReturn', aggfunc='mean')
        return html.Div([
            dcc.Graph(
                figure=px.imshow(df_pivot, title='曜日と月別のリターンのヒートマップ')
            )
        ])
    elif tab == 'tab8':
        df['MarketReturn'] = df['Close'] / df['Open'] - 1
        df['CumMarketReturn'] = df['MarketReturn'].cumsum()
        return html.Div([
            dcc.Graph(
                figure=go.Figure(
                    data=[
                        go.Scatter(x=df['Date'], y=df['CumMarketReturn'], name='Market Return'),
                        go.Scatter(x=df['Date'], y=df['CumLogReturn'], name='Cumulative Log Return')
                    ],
                    layout=go.Layout(title='マーケットオープン時とクローズ時のリターンの比較', xaxis=dict(title='Year'), yaxis=dict(title='Return'))
                )
            )
        ])
    elif tab == 'tab9':
        return html.Div([
            dcc.Graph(
                figure=px.histogram(df, x="MarketReturn", nbins=100, title='マーケットオープン時とクローズ時のリターンのヒストグラム')
            )
        ])


# アプリケーションの実行
if __name__ == '__main__':
    app.run_server(debug=True)


In [ ]:
import dash
from dash import html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import numpy as np
import dash_table
from scipy.stats import norm, shapiro
import scipy.stats as stats

# ファイルからデータを読み込む
file_path = 'sp500指数ヒストリカルデータ.csv'  # ファイルパスを適切に設定してください
df = pd.read_csv(file_path)
df['Date'] = pd.to_datetime(df['Date'])

# 指標の計算
df['LogReturn'] = np.log(df['Close'] / df['Close'].shift(1))
df['CumLogReturn'] = df['LogReturn'].cumsum()
df['PctReturn'] = df['Close'].pct_change()
df['CumPctReturn'] = (1 + df['PctReturn']).cumprod() - 1
df['SMA_20'] = df['Close'].rolling(window=20).mean()
df['SMA_100'] = df['Close'].rolling(window=100).mean()
df['weekday'] = df['Date'].dt.weekday
df['month'] = df['Date'].dt.month

# Dashアプリケーションの作成
app = dash.Dash(__name__)

# アプリケーションのレイアウト定義
app.layout = html.Div([
    dcc.Tabs(id='tabs', value='tab1', children=[
        dcc.Tab(label='基本統計量', value='tab1'),
        dcc.Tab(label='ローソク足のプロット', value='tab2'),
        dcc.Tab(label='終値と取引量のプロット', value='tab3'),
        dcc.Tab(label='リターンの分布', value='tab4'),
        dcc.Tab(label='曜日と月別のリターンのヒートマップ', value='tab7'),
        dcc.Tab(label='場中のみのリターンと場外のみのリターンの比較', value='tab8')
    ]),
    html.Div(id='tabs-content')
])

# コールバック定義：タブの選択に応じて表示するコンテンツを切り替える
@app.callback(Output('tabs-content', 'children'),
              Input('tabs', 'value'))
def render_content(tab):
 try:
    # タブ1の内容
    if tab == 'tab1':
    # 基本統計量の計算
      stats_summary = df.describe().transpose().reset_index()
      stats_summary.columns = ['Feature', 'Count', 'Mean', 'Std', 'Min', '25%', '50%', '75%', 'Max']

    # 欠損率の計算
      missing_rate = df.isnull().mean().reset_index()
      missing_rate.columns = ['Feature', 'Missing Rate']

    # 結果の表示
      return html.Div([
        html.H3('データの基本統計量'),
        dash_table.DataTable(
            data=stats_summary.to_dict('records'),
            columns=[{'name': i, 'id': i} for i in stats_summary.columns]
        ),
        html.H3('データの欠損率'),
        dash_table.DataTable(
            data=missing_rate.to_dict('records'),
            columns=[{'name': i, 'id': i} for i in missing_rate.columns]
        )
    ])

    elif tab == 'tab2':
    # ローソク足チャートの作成
      candlestick = go.Candlestick(
        x=df['Date'],
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name='ローソク足'
    )

    # レイアウトの設定
      layout = go.Layout(
        title='4本値のローソク足チャート',
        xaxis=dict(title='Date'),
        yaxis=dict(title='Price'),
        annotations=[
            dict(
                xref='paper',
                yref='paper',
                x=0.5,
                y=-0.6,
                xanchor='center',
                yanchor='bottom',
                text='一般的に金融データのAPIを使用して取得できる情報はohlcv（Open, High, Low, Close, Volume）でありこれを図示した',
                showarrow=False
            )
        ]
    )

      return html.Div([
        dcc.Graph(
            figure=go.Figure(
                data=[candlestick],
                layout=layout
            )
        )
    ])

    elif tab == 'tab3':
        return html.Div([
            dcc.Graph(
                figure=go.Figure(
                    data=[
                        go.Scatter(x=df['Date'], y=df['Close'], yaxis='y1', name='Close'),
                        go.Bar(x=df['Date'], y=df['Volume'], yaxis='y2', name='Volume', opacity=1)
                    ],
                    layout=go.Layout(
                        title='終値と取引量のプロット',
                        xaxis=dict(title='Year'),
                        yaxis=dict(title='Close'),
                        yaxis2=dict(title='Volume', overlaying='y', side='right')
                    )
                )
            )
        ])
    elif tab == 'tab4':
   # データの計算
      df['Return'] = df['Close'].pct_change()
      df['LogReturn'] = np.log(df['Close'] / df['Close'].shift(1))
      df['NormalizedClose'] = (df['Close'] - df['Close'].mean()) / df['Close'].std()

      # ヒストグラム
      hist_data_close = go.Histogram(
          x=df['NormalizedClose'].dropna(),
          opacity=0.75,
          name='標準化された終値',
          histnorm='probability density',
          nbinsx=40  # ビンの数を適切な値に設定
      )
      hist_data_return = go.Histogram(
          x=df['Return'].dropna(),
          opacity=0.75,
          name='リターン',
          histnorm='probability density',
          nbinsx=100  # ビンの数を適切な値に設定
      )
      hist_data_log_return = go.Histogram(
          x=df['LogReturn'].dropna(),
          opacity=0.75,
          name='対数リターン',
          histnorm='probability density',
          nbinsx=100  # ビンの数を適切な値に設定
      )

      hist_layout = go.Layout(
          title='標準化した終値、差分変換、対数差分変換のヒストグラム',
          xaxis=dict(title='Value'),
          yaxis=dict(title='Probability Density')
      )

      # 折れ線グラフ
      line_data_close = go.Scatter(
          x=df['Date'],
          y=df['NormalizedClose'],
          mode='lines',
          name='標準化された終値'
      )
      line_data_return = go.Scatter(
          x=df['Date'],
          y=df['Return'],
          mode='lines',
          name='リターン'
      )
      line_data_log_return = go.Scatter(
          x=df['Date'],
          y=df['LogReturn'],
          mode='lines',
          name='対数リターン'
      )

      line_layout = go.Layout(
          title='標準化した終値、差分変換、対数差分変換の時系列プロット',
          xaxis=dict(title='Date'),
          yaxis=dict(title='Value')
      )

      return html.Div([
          dcc.Graph(
              figure=go.Figure(
                  data=[hist_data_close, hist_data_return, hist_data_log_return],
                  layout=hist_layout
              )
          ),
          dcc.Graph(
              figure=go.Figure(
                  data=[line_data_close, line_data_return, line_data_log_return],
                  layout=line_layout
              )
          )
      ])
    elif tab == 'tab7':
        df_pivot = df.pivot_table(index='days of week', columns='month', values='LogReturn', aggfunc='mean')
        return html.Div([
            dcc.Graph(
                figure=px.imshow(df_pivot, title='曜日と月別のリターンのヒートマップ')
            )
        ])
    elif tab == 'tab8':
    # 前日終値と当日始値の差分を累積したリターン
        df['CumulativeReturn'] = (1 + df['Open'].pct_change()).cumprod() * df['Open'].iloc[0]

        # 同日の終値から始値を引いた市場が開いている期間のリターンの累積
        df['MarketReturn'] = (1 + (df['Close'] / df['Open'] - 1)).cumprod() * df['Close'].iloc[0]

        # グラフの作成
        cumulative_return_line = go.Scatter(
            x=df['Date'],
            y=df['CumulativeReturn'],
            mode='lines',
            name='前日終値と当日始値の差分を累積したリターン'
        )
        market_return_line = go.Scatter(
            x=df['Date'],
            y=df['MarketReturn'],
            mode='lines',
            name='市場が開いている期間のリターンの累積'
        )

        layout = go.Layout(
            title='累積リターンの比較',
            xaxis=dict(title='Date'),
            yaxis=dict(title='累積リターン（価格スケール）'),
            annotations=[
                dict(
                    xref='paper',
                    yref='paper',
                    x=0.5,
                    y=-0.3,
                    xanchor='center',
                    yanchor='bottom',
                    text='市場が開いていない期間のリターンが市場が開いている期間のリターンに比べていかに大きいかを示す',
                    showarrow=False
                )
            ]
        )

        return html.Div([
            dcc.Graph(
                figure=go.Figure(
                    data=[cumulative_return_line, market_return_line],
                    layout=layout
                )
            )
        ])


 except Exception as e:
        print(f"Error occurred: {e}")
        return html.Div([f"An error occurred: {e}"])

# アプリケーションの実行
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>